# Imports

In [49]:
import pandas as pd
from datetime import timedelta

In [50]:
df=pd.read_csv('../data/raw_data/historical_flights.csv')

In [51]:
df

,flight_id,dep_airport,dep_airport_group,arr_airport,arr_airport_group,service_type,std,sta,cancelled,atd,ata
0,WF149,HOV,B,OSL,NaN,J,2018-01-02T16:40:00.000000,2018-01-02T17:15:00.000000,0,NaN,2018-01-02T18:53:00.000000
1,WF722,OSL,NaN,MJF,D,J,2018-01-28T13:04:00.000000,2018-01-28T14:50:00.000000,0,NaN,NaN
2,WF188,FDE,A,OSL,NaN,J,2018-04-07T07:10:00.000000,2018-04-07T08:10:00.000000,0,NaN,2018-04-07T07:55:00.000000
3,WF176,HOV,B,OSL,NaN,J,2018-04-07T11:00:00.000000,2018-04-07T12:05:00.000000,0,NaN,2018-04-07T12:00:00.000000
4,WF148,HOV,B,OSL,NaN,J,2018-04-30T08:25:00.000000,2018-04-30T09:26:00.000000,0,NaN,2018-04-30T09:36:00.000000
...,...,...,...,...,...,...,...,...,...,...,...
410437,WF153,SOG,A,HOV,B,J,2025-05-03T09:25:00.000000,2025-05-03T10:00:00.000000,0,2025-05-03T09:47:00.000000,2025-05-03T10:19:00.000000
410438,WF153,BGO,NaN,SOG,A,J,2025-05-03T08:35:00.000000,2025-05-03T09:10:00.000000,0,2025-05-03T08:29:00.000000,2025-05-03T09:18:00.000000
410439,WF158,OSL,NaN,HOV,B,J,2025-05-03T14:40:00.000000,2025-05-03T15:50:00.000000,0,2025-05-03T14:35:00.000000,2025-05-03T15:39:00.000000
410440,WF721,SSJ,D,TRD,NaN,J,2025-05-03T08:50:00.000000,2025-05-03T09:45:00.000000,0,2025-05-03T08:49:00.000000,2025-05-03T09:39:00.000000


## Fjerner kansellerte fly

In [52]:
df = df[df["cancelled"] == 0].copy()

## Koverterer til datetime

In [53]:
df['std'] = pd.to_datetime(df['std'])
df['sta'] = pd.to_datetime(df['sta'])
df['atd'] = pd.to_datetime(df['atd'])
df['ata'] = pd.to_datetime(df['ata'])

## Lager intervaller for kommunikasjon

In [54]:
dep = df.dropna(subset=["atd"]).copy()
dep["start"] = dep["atd"] - pd.to_timedelta(15, "m")
dep["end"]   = dep["atd"] + pd.to_timedelta(8, "m")
dep["airport_group"] = dep["dep_airport_group"]
dep["type"] = "departure"

arr = df.dropna(subset=["ata"]).copy()
arr["start"] = arr["ata"] - pd.to_timedelta(16, "m")
arr["end"]   = arr["ata"] + pd.to_timedelta(5, "m")
arr["airport_group"] = arr["arr_airport_group"]
arr["type"] = "arrival"

intervals = pd.concat([dep, arr], ignore_index=True)
intervals = intervals.dropna(subset=["airport_group"])

## Markerer overlapp

In [55]:
def fast_overlap(group):
    events = []
    for idx, row in group.iterrows():
        events.append((row["start"], +1, idx))
        events.append((row["end"], -1, idx))
    events.sort()

    active = set()
    overlap_flags = {idx: 0 for idx in group.index}

    for time, change, idx in events:
        if change == +1:  # start
            if active:  # minst én er aktiv
                overlap_flags[idx] = 1
                for other in active:
                    overlap_flags[other] = 1
            active.add(idx)
        else:  # slutt
            active.remove(idx)

    group["target"] = group.index.map(overlap_flags)
    return group

In [56]:
results = intervals.groupby("airport_group", group_keys=False).apply(fast_overlap)


C:\Users\admin\AppData\Local\Temp\ipykernel_9456\1378367465.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = intervals.groupby("airport_group", group_keys=False).apply(fast_overlap)


In [57]:
results = results.drop(columns=["cancelled"])

In [77]:
results

,flight_id,dep_airport,dep_airport_group,arr_airport,arr_airport_group,service_type,std,sta,atd,ata,...,end,airport_group,type,target,iata_x,dep_icao,iata_y,dep_icao,iata,arr_icao
0,SK4517,MOL,B,TRF,NaN,J,2018-08-10 14:25:00,2018-08-10 17:20:00,2018-08-10 16:04:00,NaT,...,2018-08-10 16:12:00,B,departure,1,MOL,ENML,MOL,ENML,TRF,ENTO
1,WF180,FDE,A,BOO,NaN,J,2023-03-07 05:15:00,2023-03-07 07:30:00,2023-03-07 05:29:00,NaT,...,2023-03-07 05:37:00,A,departure,1,FDE,ENBL,FDE,ENBL,BOO,ENBO
2,SK4517,MOL,B,MOL,B,J,2024-01-26 15:15:00,2024-01-26 14:50:00,2024-01-26 15:45:00,NaT,...,2024-01-26 15:53:00,B,departure,0,MOL,ENML,MOL,ENML,MOL,ENML
3,WF8102,MJF,D,BOO,NaN,P,2018-01-10 12:15:00,2018-01-10 12:50:00,2018-01-10 12:10:00,2018-01-10 12:42:00,...,2018-01-10 12:18:00,D,departure,1,MJF,ENMS,MJF,ENMS,BOO,ENBO
4,WF743,MJF,D,OSL,NaN,J,2018-01-03 11:40:00,2018-01-03 13:20:00,2018-01-03 11:47:00,2018-01-03 13:36:00,...,2018-01-03 11:55:00,D,departure,1,MJF,ENMS,MJF,ENMS,OSL,ENGM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444730,WF145,SOG,A,SDN,A,J,2025-05-03 09:25:00,2025-05-03 09:55:00,2025-05-03 09:52:00,2025-05-03 10:19:00,...,2025-05-03 10:24:00,A,arrival,1,SOG,ENSG,SOG,ENSG,SDN,ENSD
444731,WF152,HOV,B,SOG,A,J,2025-05-03 06:00:00,2025-05-03 06:30:00,2025-05-03 05:57:00,2025-05-03 06:34:00,...,2025-05-03 06:39:00,A,arrival,1,HOV,ENOV,HOV,ENOV,SOG,ENSG
444732,WF153,SOG,A,HOV,B,J,2025-05-03 09:25:00,2025-05-03 10:00:00,2025-05-03 09:47:00,2025-05-03 10:19:00,...,2025-05-03 10:24:00,B,arrival,1,SOG,ENSG,SOG,ENSG,HOV,ENOV
444733,WF153,BGO,NaN,SOG,A,J,2025-05-03 08:35:00,2025-05-03 09:10:00,2025-05-03 08:29:00,2025-05-03 09:18:00,...,2025-05-03 09:23:00,A,arrival,1,BGO,ENBR,BGO,ENBR,SOG,ENSG


In [70]:
airports = pd.read_csv("../data/raw_data/airports.csv")

# Keep only relevant columns
airports = airports[["name", "icao_code", "iata_code"]]
airports = airports.rename(columns={"icao_code": "icao", "iata_code": "iata"})
airports

,name,icao,iata
0,Total RF Heliport,NaN,NaN
1,Aero B Ranch Airport,NaN,NaN
2,Lowell Field,NaN,NaN
3,Epps Airpark,NaN,NaN
4,Katmai Lodge Airport,NaN,NaN
...,...,...,...
83636,Shenyang Dongta Airport,NaN,NaN
83637,Sealand Helipad,NaN,NaN
83638,Glorioso Islands Airstrip,NaN,NaN
83639,Fainting Goat Airport,NaN,NaN


In [78]:
# lag en map fra IATA -> ICAO
iata_to_icao = dict(zip(airports["iata"], airports["icao"]))

# legg til riktig ICAO i results
results["icao_used"] = results.apply(
    lambda r: iata_to_icao.get(r["arr_airport"]) if r["type"] == "arrival"
              else iata_to_icao.get(r["dep_airport"]),
    axis=1
)

In [79]:
results

,flight_id,dep_airport,dep_airport_group,arr_airport,arr_airport_group,service_type,std,sta,atd,ata,...,airport_group,type,target,iata_x,dep_icao,iata_y,dep_icao,iata,arr_icao,icao_used
0,SK4517,MOL,B,TRF,NaN,J,2018-08-10 14:25:00,2018-08-10 17:20:00,2018-08-10 16:04:00,NaT,...,B,departure,1,MOL,ENML,MOL,ENML,TRF,ENTO,ENML
1,WF180,FDE,A,BOO,NaN,J,2023-03-07 05:15:00,2023-03-07 07:30:00,2023-03-07 05:29:00,NaT,...,A,departure,1,FDE,ENBL,FDE,ENBL,BOO,ENBO,ENBL
2,SK4517,MOL,B,MOL,B,J,2024-01-26 15:15:00,2024-01-26 14:50:00,2024-01-26 15:45:00,NaT,...,B,departure,0,MOL,ENML,MOL,ENML,MOL,ENML,ENML
3,WF8102,MJF,D,BOO,NaN,P,2018-01-10 12:15:00,2018-01-10 12:50:00,2018-01-10 12:10:00,2018-01-10 12:42:00,...,D,departure,1,MJF,ENMS,MJF,ENMS,BOO,ENBO,ENMS
4,WF743,MJF,D,OSL,NaN,J,2018-01-03 11:40:00,2018-01-03 13:20:00,2018-01-03 11:47:00,2018-01-03 13:36:00,...,D,departure,1,MJF,ENMS,MJF,ENMS,OSL,ENGM,ENMS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444730,WF145,SOG,A,SDN,A,J,2025-05-03 09:25:00,2025-05-03 09:55:00,2025-05-03 09:52:00,2025-05-03 10:19:00,...,A,arrival,1,SOG,ENSG,SOG,ENSG,SDN,ENSD,ENSD
444731,WF152,HOV,B,SOG,A,J,2025-05-03 06:00:00,2025-05-03 06:30:00,2025-05-03 05:57:00,2025-05-03 06:34:00,...,A,arrival,1,HOV,ENOV,HOV,ENOV,SOG,ENSG,ENSG
444732,WF153,SOG,A,HOV,B,J,2025-05-03 09:25:00,2025-05-03 10:00:00,2025-05-03 09:47:00,2025-05-03 10:19:00,...,B,arrival,1,SOG,ENSG,SOG,ENSG,HOV,ENOV,ENOV
444733,WF153,BGO,NaN,SOG,A,J,2025-05-03 08:35:00,2025-05-03 09:10:00,2025-05-03 08:29:00,2025-05-03 09:18:00,...,A,arrival,1,BGO,ENBR,BGO,ENBR,SOG,ENSG,ENSG


In [75]:
import requests
import pandas as pd

def get_metar(icao: str, hours: int = 3) -> pd.DataFrame:
    """
    Fetch recent METAR weather data for a given ICAO airport code.

    Parameters
    ----------
    icao : str
        ICAO airport code, e.g. "ENGM"
    hours : int
        Number of hours back to include

    Returns
    -------
    pd.DataFrame
        Weather observations with time and key fields
    """
    url = "https://aviationweather.gov/api/data/metar"
    params = {"ids": icao, "hours": hours, "format": "json"}
    headers = {"User-Agent": "JakobProject/1.0"}

    resp = requests.get(url, params=params, headers=headers)
    resp.raise_for_status()

    data = resp.json()
    if not data:  # no reports available
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["reportTime"])

    # keep only relevant fields
    keep_cols = [
        "icaoId", "time", "temp", "dewp", "wdir", "wspd",
        "visib", "altim", "wxString", "fltCat"
    ]
    return df[keep_cols]

In [76]:


# Example lookup: get ICAO for IATA codes in your list
iata_list = ["FDE", "SDN", "SOG", "HOV", "MOL"]
mapping = airports[airports["iata"].isin(iata_list)]
print(mapping)

                              name  icao iata
24230    Førde Airport, Bringeland  ENBL  FDE
24291           Molde Airport, Årø  ENML  MOL
24302  Ørsta-Volda Airport, Hovden  ENOV  HOV
24332        Sandane Airport, Anda  ENSD  SDN
24335    Sogndal Airport, Haukåsen  ENSG  SOG
